## 사용한 패키지 버전
pandas                        1.4.2
numpy                         1.21.5
seaborn                       0.11.2
beautifulsoup4                4.11.1
matplotlib                    3.5.1

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rc('font', family='Malgun Gothic')
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
data=pd.read_csv('data/loan_user_merge.csv')
data=data.drop(['Unnamed: 0'],axis=1)
data1=data.copy()

In [2]:
# object를 datetime형태로 변환
data['loanapply_insert_time']=pd.to_datetime(data['loanapply_insert_time'], format='%Y-%m-%d %H:%M', errors='raise')

#월, 일 추출
data['loanapply_month']=data['loanapply_insert_time'].dt.month
data['loanapply_day']=data['loanapply_insert_time'].dt.day
data['loanapply_week']=data['loanapply_insert_time'].dt.day_name()

# 시간으로 변경
data['loanapply_time']=data['loanapply_insert_time'].dt.time

### 입사연월 : 대출신청연도,월이랑 빼서 그전까지 근로 기간으로 대체
data['company_enter_month']=pd.to_datetime(data['company_enter_month']).dt.date
data['insert_time']=pd.to_datetime(data['insert_time']).dt.date
data['time_at_work']=data['insert_time']-data['company_enter_month']
#숫자로 변경
data['time_at_work']=data['time_at_work'].dt.days

# 외부데이터도 넣어보기
# 기준금리

def func(x) :
    if x == 3 :
        return 1.25
    elif x== 4 :
        return 1.50
    elif x==5:
        return 1.75
    elif x==6:
        return 1.75
data["기준금리"] = data["loanapply_month"].apply(lambda x : func(x))

# 생활물가지수 : 전월비

def func(x) :
    if x == 3 :
        return 1.2
    elif x== 4 :
        return 0.8
    elif x==5:
        return 1.0
    elif x==6:
        return 0.8
data["생활물가지수"] = data["loanapply_month"].apply(lambda x : func(x))

# 소비자심리지수 : 100을 기준으로 100 이하면 비관적인 소비자 심리
## 경제심리지수 : 소비자심리지수 + 기업가심리지수 : 얘는 100안내려가서 큰 효과 없을 듯! 

def func(x) :
    if x == 3 :
        return 103.2
    elif x== 4 :
        return 103.8
    elif x==5:
        return 102.6
    elif x==6:
        return 96.4
data["소비자심리지수"] = data["loanapply_month"].apply(lambda x : func(x))

# 나잇대 만들기
data['나잇대']=0
data.loc[data['birth_year']<=1963,'나잇대']='60이상'
data.loc[(data['birth_year']<=1973)&(data['birth_year']>1963),'나잇대']='50'
data.loc[(data['birth_year']<=1983)&(data['birth_year']>1973),'나잇대']='40'
data.loc[(data['birth_year']<=1993)&(data['birth_year']>1983),'나잇대']='30'
data.loc[data['birth_year']>1993,'나잇대']='20'


In [3]:
data.head()

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,birth_year,insert_time,...,personal_rehabilitation_complete_yn,loanapply_month,loanapply_day,loanapply_week,loanapply_time,time_at_work,기준금리,생활물가지수,소비자심리지수,나잇대
0,1748340,2022-06-07 13:05:41,7,191,42000000.0,13.6,NaN,430982.0,1996.0,2022-06-07,...,0.0,6,7,Tuesday,13:05:41,19150,1.75,0.8,96.4,20
1,1748340,2022-06-07 13:05:41,25,169,24000000.0,17.9,NaN,430982.0,1996.0,2022-06-07,...,0.0,6,7,Tuesday,13:05:41,19150,1.75,0.8,96.4,20
2,1748340,2022-06-07 13:05:41,2,7,24000000.0,18.5,NaN,430982.0,1996.0,2022-06-07,...,0.0,6,7,Tuesday,13:05:41,19150,1.75,0.8,96.4,20
3,1748340,2022-06-07 13:05:41,4,268,29000000.0,10.8,NaN,430982.0,1996.0,2022-06-07,...,0.0,6,7,Tuesday,13:05:41,19150,1.75,0.8,96.4,20
4,1748340,2022-06-07 13:05:41,11,118,5000000.0,16.4,NaN,430982.0,1996.0,2022-06-07,...,0.0,6,7,Tuesday,13:05:41,19150,1.75,0.8,96.4,20


# 핀다 조회수 (성별, 나이대별 일일 조회수)

In [4]:
# 핀다 조회 수 다운받아서 돌려!
lab_g_2=pd.read_excel('외부데이터/핀다/datalab (1).xlsx',header=6)
lab_g_3=pd.read_excel('외부데이터/핀다/datalab (2).xlsx',header=6)
lab_g_4=pd.read_excel('외부데이터/핀다/datalab (3).xlsx',header=6)
lab_g_5=pd.read_excel('외부데이터/핀다/datalab (4).xlsx',header=6)
lab_g_6=pd.read_excel('외부데이터/핀다/datalab (5).xlsx',header=6)
lab_b_2=pd.read_excel('외부데이터/핀다/datalab (6).xlsx',header=6)
lab_b_3=pd.read_excel('외부데이터/핀다/datalab (7).xlsx',header=6)
lab_b_4=pd.read_excel('외부데이터/핀다/datalab (8).xlsx',header=6)
lab_b_5=pd.read_excel('외부데이터/핀다/datalab (9).xlsx',header=6)
lab_b_6=pd.read_excel('외부데이터/핀다/datalab (10).xlsx',header=6)

# 데이터프레임 합칠꺼니까 열이름 바꿈
col_list=list(lab_g_2.columns)
col_list[0]='insert_time'
lab_g_2.columns=col_list
lab_g_3.columns=col_list
lab_g_4.columns=col_list
lab_g_5.columns=col_list
lab_g_6.columns=col_list

lab_b_2.columns=col_list
lab_b_3.columns=col_list
lab_b_4.columns=col_list
lab_b_5.columns=col_list
lab_b_6.columns=col_list

# 날짜 타입으로 변환
def change(data):
    data['insert_time']=pd.to_datetime(data['insert_time']).dt.date

change(lab_g_2)
change(lab_g_3)
change(lab_g_4)
change(lab_g_5)
change(lab_g_6)
change(lab_b_2)
change(lab_b_3)
change(lab_b_4)
change(lab_b_5)
change(lab_b_6)

# 20대 여성
tt=data[['insert_time','나잇대','gender']]
data_sam=pd.merge(tt,lab_g_2,on='insert_time',how='left')
only_2_girl=data_sam[(data_sam['나잇대']=='20')&(data_sam['gender']==0)]
# 30대 여성
data_sam=pd.merge(tt,lab_g_3,on='insert_time',how='left')
only_3_girl=data_sam[(data_sam['나잇대']=='30')&(data_sam['gender']==0)]
# 40대 여성
data_sam=pd.merge(tt,lab_g_4,on='insert_time',how='left')
only_4_girl=data_sam[(data_sam['나잇대']=='40')&(data_sam['gender']==0)]
# 50대 여성
data_sam=pd.merge(tt,lab_g_5,on='insert_time',how='left')
only_5_girl=data_sam[(data_sam['나잇대']=='50')&(data_sam['gender']==0)]
# 60대 이상 여성
data_sam=pd.merge(tt,lab_g_6,on='insert_time',how='left')
only_6_girl=data_sam[(data_sam['나잇대']=='60이상')&(data_sam['gender']==0)]

# 20대 남성
data_sam=pd.merge(tt,lab_b_2,on='insert_time',how='left')
only_2_boy=data_sam[(data_sam['나잇대']=='20')&(data_sam['gender']==1)]
# 30대 남성
data_sam=pd.merge(tt,lab_b_3,on='insert_time',how='left')
only_3_boy=data_sam[(data_sam['나잇대']=='30')&(data_sam['gender']==1)]
# 40대 남성
data_sam=pd.merge(tt,lab_b_4,on='insert_time',how='left')
only_4_boy=data_sam[(data_sam['나잇대']=='40')&(data_sam['gender']==1)]
# 50대 남성
data_sam=pd.merge(tt,lab_b_5,on='insert_time',how='left')
only_5_boy=data_sam[(data_sam['나잇대']=='50')&(data_sam['gender']==1)]
# 60대 이상 남성
data_sam=pd.merge(tt,lab_b_6,on='insert_time',how='left')
only_6_boy=data_sam[(data_sam['나잇대']=='60이상')&(data_sam['gender']==1)]

alla=pd.concat([only_2_boy,only_3_boy,only_4_boy,only_5_boy,only_6_boy,only_2_girl,only_3_girl,only_4_girl,only_5_girl,only_6_girl])
alla['핀다']=alla['핀다'].round() # 사람 수니까 반올림

# 원래 데이터 병합
alla=alla.sort_values(['insert_time','나잇대','gender'])
alla=alla.reset_index()
data=data.sort_values(['insert_time','나잇대','gender'])
data=data.reset_index()
data['핀다']=alla['핀다']
#data=data.drop(['insert_time','나잇대','gender'],axis=1)
#data=pd.concat([data,alla],axis=1)

# 원래 데이터 병합-램터짐
#data=pd.merge(data,alla,how='left',on=['insert_time','나잇대','gender'])


In [19]:
alla

,insert_time,나잇대,gender,대출
0,2022-03-01,20,0.0,58.0
1,2022-03-01,20,0.0,58.0
2,2022-03-01,20,0.0,58.0
3,2022-03-01,20,0.0,58.0
4,2022-03-01,20,0.0,58.0
...,...,...,...,...
13442609,2022-06-30,60이상,1.0,49.0
13442610,2022-06-30,60이상,1.0,49.0
13442611,2022-06-30,60이상,1.0,49.0
13442612,2022-06-30,60이상,1.0,49.0


## 대출 

In [3]:

lab_g_2=pd.read_excel('외부데이터/대출/datalab (1).xlsx',header=6)
lab_g_3=pd.read_excel('외부데이터/대출/datalab (2).xlsx',header=6)
lab_g_4=pd.read_excel('외부데이터/대출/datalab (3).xlsx',header=6)
lab_g_5=pd.read_excel('외부데이터/대출/datalab (4).xlsx',header=6)
lab_g_6=pd.read_excel('외부데이터/대출/datalab (5).xlsx',header=6)
lab_b_2=pd.read_excel('외부데이터/대출/datalab (6).xlsx',header=6)
lab_b_3=pd.read_excel('외부데이터/대출/datalab (7).xlsx',header=6)
lab_b_4=pd.read_excel('외부데이터/대출/datalab (8).xlsx',header=6)
lab_b_5=pd.read_excel('외부데이터/대출/datalab (9).xlsx',header=6)
lab_b_6=pd.read_excel('외부데이터/대출/datalab (10).xlsx',header=6)

# 데이터프레임 합칠꺼니까 열이름 바꿈
col_list=list(lab_g_2.columns)
col_list[0]='insert_time'
lab_g_2.columns=col_list
lab_g_3.columns=col_list
lab_g_4.columns=col_list
lab_g_5.columns=col_list
lab_g_6.columns=col_list

lab_b_2.columns=col_list
lab_b_3.columns=col_list
lab_b_4.columns=col_list
lab_b_5.columns=col_list
lab_b_6.columns=col_list

# 날짜 타입으로 변환
def change(data):
    data['insert_time']=pd.to_datetime(data['insert_time']).dt.date

change(lab_g_2)
change(lab_g_3)
change(lab_g_4)
change(lab_g_5)
change(lab_g_6)
change(lab_b_2)
change(lab_b_3)
change(lab_b_4)
change(lab_b_5)
change(lab_b_6)

# 20대 여성
tt=data[['insert_time','나잇대','gender']]
data_sam=pd.merge(tt,lab_g_2,on='insert_time',how='left')
only_2_girl=data_sam[(data_sam['나잇대']=='20')&(data_sam['gender']==0)]
# 30대 여성
data_sam=pd.merge(tt,lab_g_3,on='insert_time',how='left')
only_3_girl=data_sam[(data_sam['나잇대']=='30')&(data_sam['gender']==0)]
# 40대 여성
data_sam=pd.merge(tt,lab_g_4,on='insert_time',how='left')
only_4_girl=data_sam[(data_sam['나잇대']=='40')&(data_sam['gender']==0)]
# 50대 여성
data_sam=pd.merge(tt,lab_g_5,on='insert_time',how='left')
only_5_girl=data_sam[(data_sam['나잇대']=='50')&(data_sam['gender']==0)]
# 60대 이상 여성
data_sam=pd.merge(tt,lab_g_6,on='insert_time',how='left')
only_6_girl=data_sam[(data_sam['나잇대']=='60이상')&(data_sam['gender']==0)]

# 20대 남성
data_sam=pd.merge(tt,lab_b_2,on='insert_time',how='left')
only_2_boy=data_sam[(data_sam['나잇대']=='20')&(data_sam['gender']==1)]
# 30대 남성
data_sam=pd.merge(tt,lab_b_3,on='insert_time',how='left')
only_3_boy=data_sam[(data_sam['나잇대']=='30')&(data_sam['gender']==1)]
# 40대 남성
data_sam=pd.merge(tt,lab_b_4,on='insert_time',how='left')
only_4_boy=data_sam[(data_sam['나잇대']=='40')&(data_sam['gender']==1)]
# 50대 남성
data_sam=pd.merge(tt,lab_b_5,on='insert_time',how='left')
only_5_boy=data_sam[(data_sam['나잇대']=='50')&(data_sam['gender']==1)]
# 60대 이상 남성
data_sam=pd.merge(tt,lab_b_6,on='insert_time',how='left')
only_6_boy=data_sam[(data_sam['나잇대']=='60이상')&(data_sam['gender']==1)]

alla=pd.concat([only_2_boy,only_3_boy,only_4_boy,only_5_boy,only_6_boy,only_2_girl,only_3_girl,only_4_girl,only_5_girl,only_6_girl])
alla['대출']=alla['대출'].round() # 사람 수니까 반올림

# 원래 데이터 병합

alla=alla.sort_values(['insert_time','나잇대','gender'])
alla=alla.reset_index()
data=data.sort_values(['insert_time','나잇대','gender'])
data=data.reset_index()
data['대출']=alla['대출']


In [ ]:

lab_g_2=pd.read_excel('외부데이터/금리/datalab (1).xlsx',header=6)
lab_g_3=pd.read_excel('외부데이터/금리/datalab (2).xlsx',header=6)
lab_g_4=pd.read_excel('외부데이터/금리/datalab (3).xlsx',header=6)
lab_g_5=pd.read_excel('외부데이터/금리/datalab (4).xlsx',header=6)
lab_g_6=pd.read_excel('외부데이터/금리/datalab (5).xlsx',header=6)
lab_b_2=pd.read_excel('외부데이터/금리/datalab (6).xlsx',header=6)
lab_b_3=pd.read_excel('외부데이터/금리/datalab (7).xlsx',header=6)
lab_b_4=pd.read_excel('외부데이터/금리/datalab (8).xlsx',header=6)
lab_b_5=pd.read_excel('외부데이터/금리/datalab (9).xlsx',header=6)
lab_b_6=pd.read_excel('외부데이터/금리/datalab (10).xlsx',header=6)

# 데이터프레임 합칠꺼니까 열이름 바꿈
col_list=list(lab_g_2.columns)
col_list[0]='insert_time'
lab_g_2.columns=col_list
lab_g_3.columns=col_list
lab_g_4.columns=col_list
lab_g_5.columns=col_list
lab_g_6.columns=col_list

lab_b_2.columns=col_list
lab_b_3.columns=col_list
lab_b_4.columns=col_list
lab_b_5.columns=col_list
lab_b_6.columns=col_list

# 날짜 타입으로 변환
def change(data):
    data['insert_time']=pd.to_datetime(data['insert_time']).dt.date

change(lab_g_2)
change(lab_g_3)
change(lab_g_4)
change(lab_g_5)
change(lab_g_6)
change(lab_b_2)
change(lab_b_3)
change(lab_b_4)
change(lab_b_5)
change(lab_b_6)

# 20대 여성
tt=data[['insert_time','나잇대','gender']]
data_sam=pd.merge(tt,lab_g_2,on='insert_time',how='left')
only_2_girl=data_sam[(data_sam['나잇대']=='20')&(data_sam['gender']==0)]
# 30대 여성
data_sam=pd.merge(tt,lab_g_3,on='insert_time',how='left')
only_3_girl=data_sam[(data_sam['나잇대']=='30')&(data_sam['gender']==0)]
# 40대 여성
data_sam=pd.merge(tt,lab_g_4,on='insert_time',how='left')
only_4_girl=data_sam[(data_sam['나잇대']=='40')&(data_sam['gender']==0)]
# 50대 여성
data_sam=pd.merge(tt,lab_g_5,on='insert_time',how='left')
only_5_girl=data_sam[(data_sam['나잇대']=='50')&(data_sam['gender']==0)]
# 60대 이상 여성
data_sam=pd.merge(tt,lab_g_6,on='insert_time',how='left')
only_6_girl=data_sam[(data_sam['나잇대']=='60이상')&(data_sam['gender']==0)]

# 20대 남성
data_sam=pd.merge(tt,lab_b_2,on='insert_time',how='left')
only_2_boy=data_sam[(data_sam['나잇대']=='20')&(data_sam['gender']==1)]
# 30대 남성
data_sam=pd.merge(tt,lab_b_3,on='insert_time',how='left')
only_3_boy=data_sam[(data_sam['나잇대']=='30')&(data_sam['gender']==1)]
# 40대 남성
data_sam=pd.merge(tt,lab_b_4,on='insert_time',how='left')
only_4_boy=data_sam[(data_sam['나잇대']=='40')&(data_sam['gender']==1)]
# 50대 남성
data_sam=pd.merge(tt,lab_b_5,on='insert_time',how='left')
only_5_boy=data_sam[(data_sam['나잇대']=='50')&(data_sam['gender']==1)]
# 60대 이상 남성
data_sam=pd.merge(tt,lab_b_6,on='insert_time',how='left')
only_6_boy=data_sam[(data_sam['나잇대']=='60이상')&(data_sam['gender']==1)]

alla=pd.concat([only_2_boy,only_3_boy,only_4_boy,only_5_boy,only_6_boy,only_2_girl,only_3_girl,only_4_girl,only_5_girl,only_6_girl])
alla['금리']=alla['금리'].round() # 사람 수니까 반올림

# 원래 데이터 병합

alla=alla.sort_values(['insert_time','나잇대','gender'])
alla=alla.reset_index()
data=data.sort_values(['insert_time','나잇대','gender'])
data['금리']=alla['금리']


In [10]:
data['금리']=alla['금리']
data=data.drop(['level_0','index'],axis=1)

,level_0,index,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,...,loanapply_week,loanapply_time,time_at_work,기준금리,생활물가지수,소비자심리지수,나잇대,대출,핀다,금리
0,0,100076,475785,2022-03-01 13:59:32,25,169,23000000.0,16.9,0.0,666692.0,...,Tuesday,13:59:32,19052,1.25,1.2,103.2,20,58.0,23.0,20.0
1,1,100077,475785,2022-03-01 13:59:31,29,265,28000000.0,5.9,0.0,666692.0,...,Tuesday,13:59:31,19052,1.25,1.2,103.2,20,58.0,23.0,20.0
2,2,100078,475785,2022-03-01 13:59:32,64,254,22000000.0,13.2,0.0,666692.0,...,Tuesday,13:59:32,19052,1.25,1.2,103.2,20,58.0,23.0,20.0
3,3,100079,475785,2022-03-01 13:59:31,18,188,6000000.0,5.3,0.0,666692.0,...,Tuesday,13:59:31,19052,1.25,1.2,103.2,20,58.0,23.0,20.0
4,4,100080,475785,2022-03-01 13:59:32,1,61,11000000.0,12.9,0.0,666692.0,...,Tuesday,13:59:32,19052,1.25,1.2,103.2,20,58.0,23.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13442609,13442609,11816975,1606784,2022-06-30 19:28:48,35,29,5000000.0,14.5,NaN,399766.0,...,Thursday,19:28:48,19173,1.75,0.8,96.4,60이상,49.0,14.0,15.0
13442610,13442610,11816976,1606784,2022-06-30 19:28:40,61,92,26000000.0,19.7,NaN,399766.0,...,Thursday,19:28:40,19173,1.75,0.8,96.4,60이상,49.0,14.0,15.0
13442611,13442611,11816977,1606784,2022-06-30 19:28:39,13,262,35000000.0,15.5,NaN,399766.0,...,Thursday,19:28:39,19173,1.75,0.8,96.4,60이상,49.0,14.0,15.0
13442612,13442612,11816978,1606784,2022-06-30 19:28:40,6,36,48000000.0,17.7,NaN,399766.0,...,Thursday,19:28:40,19173,1.75,0.8,96.4,60이상,49.0,14.0,15.0


In [13]:
data.head()

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,birth_year,insert_time,...,loanapply_week,loanapply_time,time_at_work,기준금리,생활물가지수,소비자심리지수,나잇대,대출,핀다,금리
0,475785,2022-03-01 13:59:32,25,169,23000000.0,16.9,0.0,666692.0,1994.0,2022-03-01,...,Tuesday,13:59:32,19052,1.25,1.2,103.2,20,58.0,23.0,20.0
1,475785,2022-03-01 13:59:31,29,265,28000000.0,5.9,0.0,666692.0,1994.0,2022-03-01,...,Tuesday,13:59:31,19052,1.25,1.2,103.2,20,58.0,23.0,20.0
2,475785,2022-03-01 13:59:32,64,254,22000000.0,13.2,0.0,666692.0,1994.0,2022-03-01,...,Tuesday,13:59:32,19052,1.25,1.2,103.2,20,58.0,23.0,20.0
3,475785,2022-03-01 13:59:31,18,188,6000000.0,5.3,0.0,666692.0,1994.0,2022-03-01,...,Tuesday,13:59:31,19052,1.25,1.2,103.2,20,58.0,23.0,20.0
4,475785,2022-03-01 13:59:32,1,61,11000000.0,12.9,0.0,666692.0,1994.0,2022-03-01,...,Tuesday,13:59:32,19052,1.25,1.2,103.2,20,58.0,23.0,20.0


In [14]:
data=data.drop(['application_id','user_id','personal_rehabilitation_yn','personal_rehabilitation_complete_yn','loanapply_insert_time','insert_time','company_enter_month'],axis=1)

## 콜금리 크롤링


In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

date_list = []
rate_list = []

startDate = '20220301'
endDate = '20220630'

page = 1
while True:
    url = f"http://www.korcham.net/nCham/Service/EconBrief/appl/ProspectBoardList.asp?board_type=1&pageno={page}&daybt=OldNow&m_OldDate={startDate}&m_NowDate={endDate}"
    res = requests.get(url)
    soup = bs(res.text, "html.parser")

    is_last_page = len(soup.select("#contents > section.board-list > div.tablewrap > table > tbody > tr > td > b"))
    # 마지막 페이지면 종료
    if is_last_page:
        break
    else:
        table = soup.select('#contents > section.board-list > div.tablewrap > table > tbody > *')
        for i in range(len(table)):
            row = table[i].contents
            date = row[0].contents[0]
            callRate = row[1].contents[0]

            date_list.append(date)
            rate_list.append(callRate)
    
    page += 1
    print(f"{page - 1}페이지 완료")

In [ ]:
df = pd.DataFrame({'날짜': date_list, "콜금리(%)": rate_list}).sort_values(by=['날짜'])
df.to_csv('콜금리.csv', index=False)

# 콜금리 원래데이터와 병합

In [9]:
col_rate=pd.read_csv('외부데이터/콜금리.csv')
col_rate['날짜']=pd.to_datetime(col_rate['날짜'], format='%Y-%m-%d', errors='raise')
#월, 일 추출
col_rate['loanapply_month']=col_rate['날짜'].dt.month
col_rate['loanapply_day']=col_rate['날짜'].dt.day

col_rate=col_rate.drop('날짜',axis=1)
data=pd.merge(col_rate,data,how='right',on=['loanapply_month','loanapply_day'])

# 공휴일로 인한 null 전날짜로 대체
data['콜금리(%)']=data['콜금리(%)'].fillna(method='bfill')

In [ ]:
data.to_csv('include_all_merge.csv')